In [ ]:
"""
Script to visualize soil moisture time series and compute statistics.

This script loads soil moisture data previously processed and saved
by 'process_ismn_soil_moisture.ipynb' (in situ data from ISMN) and
adds an example dataset (here, ESA CCI soil moisture). It then
generates time series plots of both datasets at selected locations
and computes relevant statistics such as correlation, bias, and uRMSE.
"""

# Author: Gerard Portal
# Date: August 29, 2025  
# Contact: gerardportal@gmail.com

In [ ]:
# Import libraries

import pickle
import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
import numpy as np
from glob import glob
from datetime import timedelta,datetime
import netCDF4 as nc

In [ ]:
# Variable initialization

# Data folder path (MODIFIABLE)
hdd = 'E' # Drive letter (C,F,G,...)
path_is = hdd + ':/Path to the ISMN data generated by process_ismn_soil_moisture.ipynb/'
path_fig = hdd + ':/Path where figures are stored/'
path_cci = hdd + ':/Path to the ESA CCI SM/' # This is the CCI SM that is going to be used as an example with the in situ data from ISMN 

# Time period for which the SM maps will be generated (MODIFIABLE)
date_ini = datetime(2019,1,1)
date_fin = datetime(2022,12,31)
total_days = (date_fin-date_ini).days+1

# Reads the information of the in situ station
with open(path_is+'ISMN_SM_'+date_ini.strftime('%Y%m%d')+'_'+date_fin.strftime('%Y%m%d')+'.pckl','rb') as f:
    sm_insitu,data = pickle.load(f)

networks_names = np.unique(data[:,0]) # Constains the names of all the networks
network = networks_names[1] # Select the network(s) to be analyzed (modifiable)

# Retrieve the station information for the selected network
pos_net = np.where(data[:,0]==network)
name_stat = data[pos_net,2][0,:]
name_statU,idx_statU = np.unique(name_stat, return_index=True)
lat_stat = data[pos_net,3][0,:][idx_statU].astype(float)
lon_stat = data[pos_net,4][0,:][idx_statU].astype(float)
issm = sm_insitu[pos_net,:][0,idx_statU,:]

In [ ]:
# Definition of functions

# Function to find the nearest value in an array
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

# Read a NC
def read_NC(filename, main_var, lat_var, lon_var):
    with nc.Dataset(filename) as ds:
        # print(list(ds.variables.keys()))
        var = ds.variables[main_var][:]
        fill_value = ds.variables[main_var]._FillValue
        var[var == fill_value] = np.nan
        lat = ds.variables[lat_var][:]
        lon = ds.variables[lon_var][:] 
    return var, lat, lon

In [ ]:
# Load ESA CCI soil moisture data to compute statistics alongside in situ ISMN data
# Data source: https://catalogue.ceda.ac.uk/uuid/ff890589c21f4033803aa550f52c980c/

# Load the coordinates of the CCI soil moisture data
name_cci = glob(path_cci + '**/ESACCI-SOILMOISTURE-L3S-SSMV-COMBINED-*.nc')
_, lat_cci, lon_cci = read_NC(name_cci[0], 'sm', 'lat', 'lon')

# Time series of the CCI soil moisture
current_date = date_ini
smcci = np.full((len(lat_stat),total_days),np.nan)
while current_date<=date_fin:
    d = (current_date-date_ini).days
    name_cci = glob(path_cci + '**/ESACCI-SOILMOISTURE-L3S-SSMV-COMBINED-'+current_date.strftime('%Y%m%d')+'*.nc')
    if len(name_cci)>0:
        # Each file is opened only once per date
        with nc.Dataset(name_cci[0], mode='r') as ds:
            tmp = ds.variables['sm'][0,:,:]
            tmp = tmp.filled(np.nan)
            # The coordinates of the in situ station are used to select the closest pixel in the CCI data
            for i in range(len(lat_stat)):
                idx_lat = find_nearest(lat_cci,lat_stat[i])      
                idx_lon = find_nearest(lon_cci,lon_stat[i])     
                smcci[i,d] = tmp[idx_lat,idx_lon]
    print(current_date.strftime('%Y%m%d'),end='\r')
    current_date += timedelta(days=1)

In [ ]:
# Plot Soil Moisture Time Series and Statistics

# Study period
tmp_date_str = [date_ini + timedelta(days=i) for i in range((date_fin-date_ini).days+1)]

# Position of the text in the plot, specified as a percentage of the axes
perc_x = 10
perc_y = 92

# Y-axis limits
ylim_min = 0
ylim_max = .5

for stat_num in range(len(lat_stat)):
    fig,axs = plt.subplots(figsize=(14,3))
    axs.plot(tmp_date_str,issm[stat_num,:],color='black',label='Insitu', zorder=2)
    axs.scatter(tmp_date_str, smcci[stat_num,:],color='g',s=10,label='CCI SM', zorder=1)#cornflowerblue
    axs.set_title(f'{name_statU[stat_num]} ({network}) - lat: {lat_stat[stat_num]} lon: {lon_stat[stat_num]}')
    axs.set_ylim(ylim_min,ylim_max)
    axs.set_ylabel('Soil Moisture [m$^{3}$·m$^{-3}$]')


    # Compute statistics between in situ and CCI soil moisture
    var = smcci[stat_num,:]
    # Only dates with valid data in both datasets are used
    mask = ~np.isnan(issm[stat_num,:]) & ~np.isnan(var)
    issm_stat = issm[stat_num,:][mask]
    var = var[mask]
    corr = np.corrcoef(issm_stat,var)[0,1]        
    errors = var-issm_stat
    bias = np.mean(errors)
    urmse = np.sqrt(np.mean((errors-bias)**2))

    val = 1
    axs.text(tmp_date_str[int(total_days*perc_x/100)], ylim_max*perc_y/100, 'CCI vs in situ', color='black', fontsize=10)
    axs.text(tmp_date_str[int(total_days*perc_x/100)], ylim_max*(perc_y-7)/100, f'R = {corr:.2f}', color='black', fontsize=10)
    axs.text(tmp_date_str[int(total_days*perc_x/100)], ylim_max*(perc_y-7*2)/100, f'bias = {bias:.3f}', color='black', fontsize=10)
    axs.text(tmp_date_str[int(total_days*perc_x/100)], ylim_max*(perc_y-7*3)/100, f'uRMSE = {urmse:.3f}', color='black', fontsize=10)

    plt.grid('on',alpha=.5)
    plt.legend(loc='upper right')
    plt.savefig(path_fig+'TimeSeries_'+network+'_'+name_statU[stat_num]+'.png',dpi=300, bbox_inches='tight')
    plt.show()